# Test 11 fase 3

Entrenamiento de Mask R-CNN con dataset modificado para emular "Test 11"

### Hiperparametros
* **epoch = 300**
    * steps x epoch = 61 (lotes de imagenes)
    * batch = 5 (imágenes por lote)
* optimizador = SGD
* Funcion de perdida = SMOOTHL1LOSS
* Metrica de evaluacion = mAP (IoU >= 0.5)
* **Mini-mask shape: 28x28**
* **RPN anchor scales: (8, 16, 32, 64, 128)**
* Tasa de aprendizaje: 0.001
* **imagenes = 305**
    * entrenamiento 70% = 214
    * validacion 20% = 61
    * evaluacion 10% = 31
* etiquetas = 9140
* **resolucion = 1920 x 1080**
* etiquetas = bounding box formato VOC XML
* **numero de clases = 3 (arandano, arandano-maduro, arandano-semi-maduro)**
* **data augmentation = true**


## Comprobar directorio principal

In [1]:
!pwd && ls -l

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport/model-training
total 1956
drwxr-xr-x 4 root root   4096 Aug  7 06:10 build
drwxr-xr-x 2 root root   4096 Aug  7 06:10 dist
drwxr-xr-x 2 root root   4096 Aug  8 03:15 mask_rcnn.egg-info
drwxr-xr-x 4 root root   4096 Sep 18 19:33 mrcnn
drwxr-xr-x 3 root root   4096 Sep 15 02:27 old
-rw-r--r-- 1 root root  67789 Sep 16 23:55 test_11.ipynb
-rw-r--r-- 1 root root  92388 Sep 23 06:47 test_11_2_fase_2_corregido.ipynb
-rw-r--r-- 1 root root  88232 Sep 17 07:27 test_11_fase_2-Copy1.ipynb
-rw-r--r-- 1 root root  67792 Sep 17 08:36 test_11_fase_2.ipynb
-rw-r--r-- 1 root root  87782 Sep 21 20:06 test_11_fase_2_corregido.ipynb
-rw-r--r-- 1 root root  25842 Oct  8 07:11 test_11_fase_3.ipynb
-rw-r--r-- 1 root root 350337 Sep 15 00:38 test_5-fase_2.ipynb
-rw-r--r-- 1 root root 163924 Sep 18 23:12 test_5_corregido.ipynb
-rw-r--r-- 1 root root 350144 Sep 17 08:42 test_5_fase_2.ipynb
-rw-r--r-- 1 root root 102855 Sep 20 18:56 test_5_fase_2_corregido.ipynb
-rw

# Importar bibliotecas

In [1]:
# bibliotecas basicas
import os
from os import listdir
import sys
import json
import datetime

#sys.path.append("/tf/PT_JoseVeloso/Mask_RCNN-master/")

# bibliotecas avanzadas 
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# bibliotecas mask rcnn 
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# biblioteca matplotlib 
import matplotlib.pyplot as plt

# bibliotecas numpy 
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# bibliotecas keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img   #keras.preprocessing.image tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# ignorar alertas de elementos que seran descontinuados
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
#plt.show()

import imgaug.augmenters as iaa

# Fase 2 - Entrenamiento con dos clases y etiquetas de Bounding Box

En este entremamiento se utiliza un conjunto de datos simple con imágenes etiquetadas con cuadros delimitadores y una clase llamada 'Daño'. En la siguiente sección se encuentra el código para el entrenamiento del modelo. Se incluyen comentarios para describir mejor el flujo del programa.

In [2]:
class DatasetArandanos(Dataset):
    
    # la funcion load_dataset es usada para cargar el dataset de entrenamiento y test
    def load_dataset(self, dataset_dir, is_train=True):
        
        # se agrega una clase que se necesita para clasificar, en este caso arandano
        # self.add_class('source', 'class id', 'class name')
        self.add_class("dataset", 1, "arandano")
        self.add_class("dataset", 2, "arandano-maduro")
        self.add_class("dataset", 3, "arandano-semi-maduro")
        #self.add_class("dataset", 4, "arandano-semi")
        
        # se concatena dataset_dir con /images y /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train sera Verdadero si se esta entrenando el modelo y Falso si se esta testeando el modelo
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4] # se usa para omitir los últimos 4 caracteres: '.jpg' (en class_id.jpg)
            
            # si is_train es Verdadero se omiten todas las imágenes con id mayor que e iguales a 
            # aproximadamente el 70% del conjunto de datos es para entrenamiento
            if is_train and int(image_id) >= 40720 :
                #print("image_id: ", image_id)
                continue
             
            # si is_train no es Verdadero se omiten todas las imágenes con id menores a
            if not is_train and int(image_id) < 40720:
                continue
            
            # se declara la ruta de la imagen y la ruta de las etiquetas 
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # usando la función add_image se pasan image_id, image_path y ann_path para que la 
            # imagen actual se agregue al conjunto de datos para entrenamiento o prueba
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # funcino usada para extraer bouding boxes desde archivos etiquetados 
    def extract_boxes(self, filename):

        # se puede ver en las imágenes que estan etiquetadas, como se extraen los valores de ancho, alto y bndbox

        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>

        # </annotation>
        
        # para analizar los archivos .xml
        tree = ElementTree.parse(filename)
        
        # para obtener la raíz del archivo xml
        root = tree.getroot()
        
        # se agregan todas las coordenadas x, y en boxes para todas las instancias de un objeto
        boxes = list()
        
        # se encuentran todos los atributos con el nombre bndbox que existan para cada ground truth en la imagen
        for box in root.findall('.//object'):
            name = box.find('name').text
            xmin = int(box.find('./bndbox/xmin').text)
            ymin = int(box.find('./bndbox/ymin').text)
            xmax = int(box.find('./bndbox/xmax').text)
            ymax = int(box.find('./bndbox/ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
                        
            #quita todas las imagenes no etiquetadas
            if name=='arandano' or name=='arandano-maduro'or name=='arandano-semi-maduro':
                boxes.append(coors)

        # extraer ancho y alto de la imagen
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # retorna boxes-> list, width-> int y height-> int 
        return boxes, width, height
    
    # esta función llama al método extract_boxes y se usa para cargar una máscara para cada instancia en una imagen 
    # devuelve una máscara booleana con las siguientes dimensiones ancho * alto * instancias
    def load_mask(self, image_id):
        
        # info apunta al image_id actual 
        info = self.image_info[image_id]
        
        # se obtiene la ruta de anotación de image_id que es dataset_dir/annots/image_id.xml 
        path = info['annotation']
        
         # se llama al método extract_boxes (arriba) para obtener bndbox del archivo .xml
        boxes, w, h = self.extract_boxes(path)
        
        # se crea una cantidad de len(boxes) de mascaras de alto 'h' y ancho 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        class_ids = list()
        
        ## se recorren todos los boxes y generamos máscaras (máscara de bndbox) y class id para cada instancia
        # las máscaras tendrán forma rectangular ya que hemos usado bndboxes para etiquetas
        # por ejemplo: si 2.jpg tiene tres objetos, tendremos las siguientes máscaras y class_ids.
        
        # 000000000 000000000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000000000 022200000 000000000 000000000
        #    1         2          3         1<- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            
            # box[4] will have the name of the class for a particular damage
            if (box[4] == 'arandano'):
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('arandano'))
            elif(box[4] == 'arandano-maduro'):
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('arandano-maduro')) 
            elif(box[4] == 'arandano-semi-maduro'):
                masks[row_s:row_e, col_s:col_e, i] = 3
                class_ids.append(self.class_names.index('arandano-semi-maduro'))

                
        # retorna mascaras y class_ids como arreglo
        return masks, asarray(class_ids, dtype='int32')
    
    # esta funciones toma el image_id y retorna la ruta de la imagen 
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [3]:

# damage configuration class, you can change values of hyper parameters here
class ConfigArandanos(Config):

    # nombre de la configuracion
    NAME = "arandano_cfg_test_11_fase_3_"    
    
    # clase arandano + clase background + 4 clases
    NUM_CLASSES = 1 + 3
    
    # pasos por epoch y confianza minima    # STEPS_PER_EPOCH = cantidad de lotes/batchs
    STEPS_PER_EPOCH = 61  # por epoch se entrenaran 61 lotes de 5 imagenes, dataset = 305

    # tasa de aprendizaje y momentum
    LEARNING_RATE=0.001
    LEARNING_MOMENTUM = 0.8
    
    # penalización de regularización
    WEIGHT_DECAY = 0.0001
    
    # el tamaño de la imagen está controlado por este parámetro
    IMAGE_MIN_DIM = 512
    
    # pasos de validación
    VALIDATION_STEPS = 50
    
    # número de regiones de interés generadas por imagen
    Train_ROIs_Per_Image = 200
    
    # escala de anclas RPN y proporciones (ratios) para encontrar la ROI
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)    # Longitud del lado del ancla cuadrada, en píxeles 
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]   # Proporciones de anclas por cada celda (ancho/alto). Un valor de 1 representa un ancla cuadrada y 0,5 es un ancla ancha 

    #DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0    
    DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

    IMAGES_PER_GPU = 2
    
    MINI_MASK_SHAPE = (28, 28)
    
ConfigArandanos().display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DEVICE                         /gpu:0
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.8
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE 

### Entrenamiento

In [5]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [4]:
pwd

'g:\\Mi unidad\\UBB\\PT\\Mask_R-CNN\\Mask R-CNN\\entrenamiento\\model-training'

In [6]:
#pesos = 'arandano_cfg_test_6_fase_2_2_20220920T0813/mask_rcnn_arandano_cfg_test_6_fase_2_2__0068.h5' 
pesos = 'arandano_cfg_test_11_fase_2_2_20220921T2007/mask_rcnn_arandano_cfg_test_11_fase_2_2__0120.h5'
pesos = 'entrenamiento/arandano_cfg_test_11_fase_2_2_20220921T2007/mask_rcnn_arandano_cfg_test_11_fase_2_2__0120.h5'

conjunto_datos = 'customImages/test_11_fase_3'
'../../../dataset_v2/test_11_fase_3'

In [ ]:
# cargar dataset de entrenamiento
train_set = DatasetArandanos()
train_set.load_dataset(conjunto_datos, is_train=True)
train_set.prepare()

# cargar dataset de test 
test_set = DatasetArandanos()
test_set.load_dataset(conjunto_datos, is_train=False)
test_set.prepare()

# preparar la configuración llamando a la clase de configuración definida por el usuario
config = ConfigArandanos()

# definir el modelo
with tf.device(config.DEVICE):
    model = MaskRCNN(mode='training', model_dir='./', config=config)

# cargar pesos del modelo 
weights_path = pesos

# cargar los pesos del modelo
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

#augmentation = iaa.Sometimes(0.5, iaa.Fliplr(0.5), iaa.Flipud(0.5))

augmentation = iaa.Sequential([
    iaa.Flipud(0.5),
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.2)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    )],  
    random_order=True)

#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50, baseline=2.5, min_delta=1)
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50, min_delta=1)

es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=50,
    verbose=1,
    mode="min",
    baseline=2.5,
    restore_best_weights=True
)

# iniciar el entrenamiento del modelo
# layers = head or all
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=300, layers='all', augmentation=augmentation)
#model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=300, layers='all', augmentation=augmentation, custom_callbacks=es)


Starting at epoch 0. LR=0.001

Checkpoint Path: ./arandano_cfg_test_11_fase_3_20221008T0712/mask_rcnn_arandano_cfg_test_11_fase_3__{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/300


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/ROI_1/GatherV2_2_grad/Reshape_1:0", shape=(6000,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/ROI_1/GatherV2_2_grad/Reshape:0", shape=(6000, 4), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/ROI_1/GatherV2_2_grad/Cast:0", shape=(2,), dtype=int32, device=/device:GPU:0))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/ROI_1/GatherV2_3_grad/Reshape_1:0", shape=(6000,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/ROI_1/GatherV2_3_grad/Reshape:0", shape=(6000, 4), dtype=float32), dense_shape=Ten

61/61 [==============================] - 603s 10s/step - batch: 30.0000 - size: 2.0000 - loss: 4.2012 - rpn_class_loss: 0.4839 - rpn_bbox_loss: 1.7585 - mrcnn_class_loss: 0.3752 - mrcnn_bbox_loss: 1.0662 - mrcnn_mask_loss: 0.5174 - val_loss: 2.9249 - val_rpn_class_loss: 0.4580 - val_rpn_bbox_loss: 0.9541 - val_mrcnn_class_loss: 0.2059 - val_mrcnn_bbox_loss: 0.7249 - val_mrcnn_mask_loss: 0.5821
Epoch 2/300
61/61 [==============================] - 471s 8s/step - batch: 30.0000 - size: 2.0000 - loss: 2.3464 - rpn_class_loss: 0.1057 - rpn_bbox_loss: 0.8720 - mrcnn_class_loss: 0.1681 - mrcnn_bbox_loss: 0.6606 - mrcnn_mask_loss: 0.5401 - val_loss: 2.7081 - val_rpn_class_loss: 0.4566 - val_rpn_bbox_loss: 0.8525 - val_mrcnn_class_loss: 0.2344 - val_mrcnn_bbox_loss: 0.6145 - val_mrcnn_mask_loss: 0.5501
Epoch 3/300
61/61 [==============================] - 428s 7s/step - batch: 30.0000 - size: 2.0000 - loss: 2.1908 - rpn_class_loss: 0.1017 - rpn_bbox_loss: 0.8278 - mrcnn_class_loss: 0.1490 - mrcn